In [6]:
# 9-1 python中的迭代协议
#什么是迭代协议
#迭代器是什么？ 迭代器是访问集合内元素的一种方式， 一般用来遍历数据
#迭代器和以下标的访问方式不一样， 迭代器是不能返回的（即 不可以通过像list那样下标的方式来得到对应的返回值）, 迭代器提供了一种惰性访问数据的方式

# 生成器背后的原理也会是迭代器，    Iterator继承了Iterable, Generation又是继承的Iterator 实现了Iterator的__next__方法
# 惰性方式数据 在访问数据的时候才去计算或获取数据
#[] list , __iter__ 迭代协议  实现了该方法的就是可迭代类型    而迭代器还必须实现 __next__方法

'''
class Iterator(Iterable):

    __slots__ = ()

    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration

    def __iter__(self):    # Iterator 中已经重载了父类Iterable的iter方法
        return self  
'''
# Iterator 继承自Iterable,Iterator有两个必须要实现的方法 一个是自身的 __next__  是返回下个数据的  还有一个是Iterable 的方法 __iter__用来返回迭代器

from collections.abc import Iterable, Iterator
a = [1,2]
iter_rator = iter(a)    #iter(iterable) -> iterator 将迭代对象 生成一个迭代器
print (isinstance(a, Iterable))     # list是可迭代对象却不是迭代器
print (isinstance(iter_rator, Iterator))

True
True


In [9]:
# python 通过是否实现了一些共有的魔法函数（即协议） 而判断某个类是否属于某种类   所以只要实现了相应的魔法函数，就会具有相应类型的一些特性

class a:
    def __iter__(self):
        pass
a = a()    
print (isinstance(a, Iterable)) 



True


In [ ]:
# 9-2 什么是迭代器和可迭代对象

In [5]:
class Company(object):
    def __init__(self, employee_list):
        self.employee = employee_list

    def __getitem__(self, item):
        return self.employee[item]
    
if __name__ == "__main__":
    company = Company(["tom", "bob", "jane"])
    # 为什么可以for循环，其实内部for循环的时候是会尝试调用iter() 即iter(company)  会首先查找内部是否有__iter__方法，没有的话会利用__getitem__创建一个迭代器Iterator，
    for item in company:   
        print(item)
        
#     iter(company) 得到了一个迭代器对象<iterator object at 0x0000028C4EE34710>

tom
bob
jane


In [9]:
from collections.abc import Iterator

class Company(object):
    """
    事实上我们也可以自己在类中设计实现 __next__方法，而使其成为一个迭代器对象
    但是迭代器的设计规范和设计模式要求 我们应该通过自定义一个迭代器类来实现，将index的维护放到单独的迭代器类中维护，而不要放在当前类中

    比如像内置的list类型，也没有实现 __next__  而是单独设计了一个迭代器 在__iter__中进行调用

    """

    def __init__(self, employee_list):
        self.employee = employee_list

    def __iter__(self):  # 返回一个迭代器对象
        return MyIterator(self.employee)   # 使用我们自己创建的迭代器类生成一个迭代器实例对象

    # def __getitem__(self, item):
    #     return self.employee[item]


class MyIterator(Iterator):                    #  也可以不使用继承的方式而是直接使用__iter__魔法函数，因为Python是基于协议的
    """
    自定义迭代器，因为继承自Iterator，内部已经实现了__iter__方法
    实际上， 如果只是将这个类作为工具类，为其他地方的__iter__提供调用，那么不需要在本类中也实现__iter__方法
    但最好写上，或者直接继承Iterator类（l里面写好了__iter__方法）为了防止将自身作为可迭代对象使用，此时不能没有__iter__方法
    """
    def __init__(self, employee_list):
        self.iter_list = employee_list
        self.index = 0  # 需要维护内部变量，用来标识读取到的位置
        
#     def __iter__(self):
#         print("我被调用了")
#         return self

    def __next__(self):      # 是不会传递索引的   __next__能从任意位置读数据
        #真正返回迭代值的逻辑
        try:
            word = self.iter_list[self.index]
        except IndexError:  # 超出索引
            raise StopIteration # 迭代器到结尾的时候要主动抛出StopIteration异常，该异常能够被for语句自动处理，不会再抛出     这里实际做的是 一个异常的转换
        self.index += 1
        return word

if __name__ == "__main__":
    print(issubclass(MyIterator, Iterator))
    company = Company(["tom", "bob", "jane"])
    # 方式一： 得到生成器对象
#     my_itor = iter(company)
#     while True:
#         try:
#             print (next(my_itor))
#         except StopIteration:
#             break
    # 方式二：利用for训话自动找__iter__使用iter()包装
    for item in company:
        print(item)


True
tom
bob
jane


In [11]:
# 9-3 生成器函数的使用                      含有yield关键字的函数就是生成器

#生成器函数，函数里只要有yield关键字
def gen_func():
    yield 1 
    yield 2
    yield 3

# 调用生成器 返回的是生成器对象，是在Python编译字节码的时候就产生的
gen = gen_func()
print(gen)  # 得到的是一个生成器对象
print("---"*30)
# 生成器对象实际上也是实现了迭代器协议（__iter__）的，所以直接for循环就能遍历值      
for value in gen:
    print(value)

<generator object gen_func at 0x0000029CAD0BDB10>
------------------------------------------------------------------------------------------
1
2
3


In [10]:
#惰性求值， 延迟求值提供了可能

# 斐波拉契数列
# 斐波拉契 1 1 2 3 5 8 13 21 34 55
# 递归：
def fib(index):
#     if index <= 2:
#         return 1
#     else:
#         return fib(index-1) + fib(index-2)
    return 1 if index <= 2 else fib(index-1) + fib(index-2)
print(fib(10))

55


In [27]:
# 缺点是 当数很多时，非常消耗内存
def fib2(index):
    re_list = []
    n,a,b = 0,0,1
    while n<index:
        re_list.append(b)
        a,b = b, a+b
        n += 1
    return re_list
print(fib2(10))

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


In [34]:
# 使用生成器   不消耗内存，不会增加内存，因为只有在计算的时候才会增加值
def gen_fib(index):
    n,a,b = 0,0,1
    while n<index:
        yield b
        a,b = b, a+b
        n += 1
        
for data in gen_fib(10):
    print(data)
# next(gen_fib(10))

1
1
2
3
5
8
13
21
34
55


In [ ]:
# 9-4 python是如何实现生成器的

生成器对象一旦产生 可以在任何地方对其进行调用，暂停
生成器内部实现了对函数的暂停（内部有变量f_lasti记录函数运行到的位置（下次继续运行的时候，就知道从什么位置开始运行了），f_locals记录生成器返回的值） 而不是像普通函数那样一次性运行完，

In [37]:
# 9-5 生成器在UserList中的应用

a = [1, 2, 3]
for i in a:
    print(i)

# 能够for进行遍历，这样实际是因为for循环的时候，执行 for iter() 去找类中是否有__iter__如果没有会退化到找 __getitem__方法    使用iter() 就是去执行类的__iter__方法

1
2
3


In [ ]:
from collections import UserList
# UuserList的父类的父类Sequence实现了__iter__方法
    def __iter__(self):
        i = 0
        try:
            while True:
                v = self[i]
                yield v
                i += 1
        except IndexError:
            return
# 从下面的检测总可以看出，一直调用多次

In [131]:
from collections import UserList
a = UserList([1,2,3,4])
print(a.__iter__())   # 得到的是生成器对象   调用iter()这个方法，就是相当于执行了__iter__
print(iter(a))
print(iter(a))                                                                              # 多次调用统一个方法返回的对象应该是不同的，但是生成器是相同的，__iter__就是一个生成器，之前解释了原理，生成器是会记录本身位置以及值的
b = a.__iter__()   #此处就是执行了一个生成器，得到b 是一个生成器对象
next(b) # 或者是 b.__next__()

<generator object Sequence.__iter__ at 0x0000029CAD5CFB88>
<generator object Sequence.__iter__ at 0x0000029CAD5CFB88>
<generator object Sequence.__iter__ at 0x0000029CAD5CFB88>


1

In [124]:
b = [12,3,54]
print(b.__iter__())   # 得到的是迭代器对象
print(iter(b))      # 原本实现了__iter__方法的类实例，只要调用__iter__就能得到生成器对象，该对象就可以使用next() 或者 调用__next__()
                    # 而可见，调用iter()这个方法，就是相当于执行了__iter__，list中会得到相应的生成器对象，从结果可知 都是list_iterator object 列表迭代器对象
print(iter(b))
print(iter(b))      # list每次调用得到的迭代器对象不是同一个 就应该是不同的，因为都是重新调用了__iter__，得到的是重新生成的的迭代器对象

a = iter(b)
c = iter(b)
print(a,c)

bb = b.__iter__()
print(next(bb))  # 12


<list_iterator object at 0x0000029CAD575128> <list_iterator object at 0x0000029CAD5803C8>
12


In [130]:
class a():
#     def __init__(self, a):
#         self.a = a
        
    def aa(self):
        a = B()
        return a

    
class B():
    pass


print(a())
print(a())
# print(a())
print(a())
# print(a())
# print(a())
# aaa = a(1)
# print("a的两次调用："+"---"*30)
# print(aaa.aa())
# print(aaa.aa())
# bbb = a(1)
# print("b的一次调用："+"---"*30)
# print(bbb.aa())
# print("---"*30)
# print(aaa)
# print(bbb)                                                
'''
python 似乎会在一个对象生成的次数过多的时候，（且生成的对象都没有调用），会自动使用之前得到的对象
'''

1. <font color='red' size='3'>iter()方法就是调用对象的`__iter__`魔法函数，并将`__iter__`的返回值作为自己的返回值</font>  此外，UserList和list的`__iter__`不同，UserList是生成器原理，而list是迭代器原理
2. 含有yield关键字的函数就是生成器，生成器实际上是特殊的一种迭代器，即完全可以想使用迭代器一样使用生成器


In [20]:
# 注意：当使用迭代器迭代一个可变对象的时候，对可变对象的操作会影响到迭代器的值， 因为序列的迭代器只是记录当前到达了序列的第几个元素，
# 所以如果迭代的过程中改变了序列的元素，更新会立即反映到所迭代的条目上

a = list([1,2,3,4,5])
b = iter(a)
print(next(b))
print(next(b))
a.append(5)
print(a)
print(next(b))
print(next(b))
print(next(b))
print(next(b))
a.append(56)
print(next(b))

1
2
[1, 2, 3, 4, 5, 5]
3
4
5
5
56


In [24]:
# 9-6 生成器如何读取大文件

'''
读取500G的文件，若文件是一行一行的时候，可以采用下面的方式
f = open()
方式一：
for line in f:
  line.read()
方式二：
f.readlines()
若文件只有一行的时候，这样就不行了，占用内存


read
'''
'''
处理读取大文件，或者只有一行的文件的处理方法
'''

#500G, 特殊 一行
def myreadlines(f, newline):     # 参数为句柄 和 分隔符
    buf = ""  # 缓存
    while True:
        while newline in buf:   # 缓存中是否有分隔符newline   第一次读取的时候不成立    如果读取的4096数据过长，有多个分隔符，则在循环中都会处理完，使得缓存中没有分隔符
            '''
            while循环保证了缓存中不会存储超出读取的4096的两倍长，即每次在读4096个数据的时候，缓存中不会有超出去4096长的数据
            如果使用
            '''
            pos = buf.index(newline)  # 找到分隔符的位置
            yield buf[:pos]  # 将缓存中 分割符之前的数据返回
            buf = buf[pos + len(newline):] # 更新缓存  去除返回的数据，将剩下的数据重新放到缓存中
        chunk = f.read(4096)   # 读取4096长的字符  read内部会维护这个偏移量，不需要我们进行设置

        if not chunk:   # 没有值的时候成立 即已经读取完整
            '''
            边界条件处理，即读不到数据了，就将缓存中剩余的内容全部返回（缓存中肯定没有分隔符，以及长度不大于4096）
            用于处理结尾没有分割符而滞留在缓存中的数据
            '''
            #说明已经读到了文件结尾
            yield buf   # 将最后剩余的内容返回
            break       # 结束整个循环过程
        buf += chunk  # 拼接进缓存

with open("input.txt") as f:
    for line in myreadlines(f, "{|}"):
        print (line)



Prior to beginning tutoring sessions
, I ask new students to fill
 out a brief self-assessment
 where they rate their
 understanding of various Python concepts. Some topics ("control flow with if/else" or "defining and using functions") are understood by a majority of students before ever beginning tutoring. There are a handful of topics, however, that almost all
 students report having no knowledge or very limited understanding of. Of these

